In [106]:
import xml.etree.ElementTree as Xet
import pandas as pd
import numpy as np
  

def get_text_from_row(row,name):
    val = row.find(name)
    if val != None:
        val = val.text
    return val

#I used this data source from 20.05.2022 https://www.marktstammdatenregister.de/MaStR/Datendownload
#convert xml files to csv
def generate_csv(energy,name):

    PATH = "data/mastr/Einheiten" + name + ".xml"

    cols = ["energy","land","bundesland", "landkreis", "gemeinde", "laengengrad","breitengrad","registrierungsdatum","inbetriebnahmedatum","datumEndgueltigeStilllegung","bruttoleistung","nameEinheit","betriebsstatus","energietraeger"]
    rows = []
  
    # Parsing the XML file
    xmlparse = Xet.parse(PATH)
    root = xmlparse.getroot()
    for i in root:

        land = get_text_from_row(i,"Land")
        bundesland = get_text_from_row(i,"Bundesland")
        landkreis = get_text_from_row(i,"Landkreis")
        gemeinde = get_text_from_row(i,"Gemeinde")
        laengengrad = get_text_from_row(i,"Laengengrad")
        breitengrad = get_text_from_row(i,"Breitengrad")
        Registrierungsdatum = get_text_from_row(i,"Registrierungsdatum")
        Inbetriebnahmedatum = get_text_from_row(i,"Inbetriebnahmedatum")
        DatumEndgueltigeStilllegung = get_text_from_row(i,"DatumEndgueltigeStilllegung")
        bruttoleistung = get_text_from_row(i,"Bruttoleistung")
        nettonennleistung = get_text_from_row(i,"Nettonennleistung")
        betriebsstatus = get_text_from_row(i,"EinheitBetriebsstatus") 
        energietraeger= get_text_from_row(i,"Energietraeger")
        #35 = in betrieb
        #38 = endgültig stillgelegt
        #31 = in planung
        #37 = vorübergehend stillgeleft
        nameEinheit = get_text_from_row(i,"NameStromerzeugungseinheit")

    
        rows.append({"energy":energy,
                    "land": land,
                    "bundesland": bundesland,
                    "landkreis": landkreis,
                    "gemeinde": gemeinde,
                    "laengengrad":laengengrad,
                    "breitengrad":breitengrad,
                    "registrierungsdatum":Registrierungsdatum,
                    "inbetriebnahmedatum":Inbetriebnahmedatum,
                    "datumEndgueltigeStilllegung":DatumEndgueltigeStilllegung,
                    "nameEinheit":nameEinheit,
                    "bruttoleistung": bruttoleistung,
                    "nettonennleistung":nettonennleistung,
                    "betriebsstatus":betriebsstatus,
                    "energietraeger":energietraeger})
    
    df = pd.DataFrame(rows, columns=cols)
    OUTPUT_PATH = 'data/mastr/output/' + name + '.csv'
    df.to_csv(OUTPUT_PATH)
  

In [ ]:
#I did not include Gas, Speicher Units and Consumption Units

ENG = "wind"
NAME = "Wind"
generate_csv(ENG,NAME)

In [ ]:
NAME = "wasser"
OUTPUT_PATH = 'data/mastr/output/' + NAME + '.csv'
df = pd.read_csv(OUTPUT_PATH)
df

In [ ]:
#combine data to one csv about power

read_files=["Solar_1.csv","Solar_10.csv","Solar_11.csv","Solar_12.csv","Solar_13.csv","Solar_14.csv","Solar_15.csv","Solar_16.csv","Solar_17.csv","Solar_18.csv","Solar_19.csv","Solar_2.csv","Solar_20.csv","Solar_21.csv","Solar_22.csv","Solar_23.csv","Solar_24.csv","Solar_3.csv","Solar_4.csv","Solar_5.csv","Solar_6.csv","Solar_7.csv","Solar_8.csv","Solar_9.csv","Verbrennung.csv","Wasser.csv","Wind.csv","Biomasse.csv","GeoSolarthermieGrubenKlaerschlammDruckentspannung.csv","Kernkraft.csv"]

all_df = []
for file in read_files:
    path = "data/mastr/" + file
    df = pd.read_csv(path)
    all_df.append(df)

concat_table = pd.concat(all_df)
concat_table.to_csv("data/mastr/powerProductionGermany.csv")


In [103]:
#please download this file from our google drive 

df_org = pd.read_csv("data/powerProductionGermany.csv")
df_org

,Unnamed: 0,Unnamed: 0.1,energy,land,bundesland,landkreis,gemeinde,laengengrad,breitengrad,registrierungsdatum,inbetriebnahmedatum,datumEndgueltigeStilllegung,bruttoleistung,nameEinheit,betriebsstatus,energietraeger
0,0,0,solar,84,1408.0,Region Hannover,Seelze,NaN,NaN,2019-01-31,2017-11-17,NaN,6.00,Photovoltaik Dedensen,35,2495
1,1,1,solar,84,1409.0,Essen,Essen,NaN,NaN,2019-01-31,2007-06-01,NaN,3.24,Hausdach,35,2495
2,2,2,solar,84,1408.0,Osnabrück,Merzen,NaN,NaN,2019-01-31,2014-07-17,NaN,8.30,Fronius Symo,35,2495
3,3,3,solar,84,1406.0,Hamburg,Hamburg,NaN,NaN,2019-01-31,2008-08-27,NaN,2.92,Hausdach,35,2495
4,4,4,solar,84,1403.0,Bayreuth,Hollfeld,NaN,NaN,2019-01-31,2013-06-18,NaN,5.50,"Anlage-1-PV-5,5kwp",35,2495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515865,1,1,kernkraft,84,1411.0,Steinburg,Brokdorf,9.344720,53.850830,2020-06-25,1986-10-14,2021-12-31,1480000.00,Brokdorf,38,2494
2515866,2,2,kernkraft,84,1403.0,Landshut,Essenbach,12.293150,48.605600,2020-06-25,1988-01-22,NaN,1485000.00,Isar 2,35,2494
2515867,3,3,kernkraft,84,1403.0,Günzburg,Gundremmingen,10.404743,48.513566,2020-06-25,1984-11-02,2021-12-31,1344000.00,Gundremmingen C,38,2494
2515868,4,4,kernkraft,84,1408.0,Emsland,Lingen (Ems),7.324140,52.472897,2020-06-25,1988-06-20,NaN,1406000.00,Emsland A,35,2494


In [110]:
df = df_org.copy();

#only producers currently producing
df = df[df.betriebsstatus == 35]

# bruttoleistung of different energy producers per location
# Create pivot table
pivot_table_df = pd.pivot_table(
    df,
    index='energy',
    columns='landkreis',
    values='bruttoleistung',
    aggfunc=np.sum,
    margins=False
)

# Sort table
#pivot_table_df.sort_values(
#    by=['All'],
#    inplace=True,
#    ascending=False
#)

pivot_table_df = pivot_table_df.rename_axis(None).transpose().reset_index(level=0).fillna(0)
pivot_table_df

pivot_table_df.to_csv("data/powerPerLandkreisAndEnergy.csv")

#fig = px.bar(pivot_table_df, x="landkreis", y=['Solare Strahlungsenergie','Speicher','Wind','Erdgas','Biomasse','andere Gase','Mineralölprodukte','Wasser'], title="Energy Production per Category in Barnim")
#fig.show()